In [ ]:
# Full Lung Cancer Assistant App
# Using PyTorch model + Local LLM via LM Studio + Streamlit GUI

import os
import streamlit as st
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
from datetime import datetime
import pandas as pd

# -------- Configuration --------
MODEL_PATH = "lung_model.pth"
IMG_SIZE = 224

data_log_file = "prediction_log.csv"

# -------- Define CNN Model --------
class LungCNN(nn.Module):
    def __init__(self):
        super(LungCNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 56 * 56, 128), nn.ReLU(),
            nn.Linear(128, 3)
        )

    def forward(self, x):
        x = self.conv(x)
        return self.fc(x)

# -------- Image Transform --------
transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# -------- Prediction --------
def predict_image(image: Image.Image):
    model = LungCNN()
    model.load_state_dict(torch.load(MODEL_PATH, map_location=torch.device('cpu')))
    model.eval()
    input_tensor = transform(image).unsqueeze(0)
    with torch.no_grad():
        outputs = model(input_tensor)
        _, pred = torch.max(outputs, 1)
    return ["benign", "malignant", "normal"][pred.item()]

# -------- Local LLM Integration (No OpenAI) --------
def ask_local_llm(prompt):
    try:
        import requests
        response = requests.post("http://localhost:1234/v1/chat/completions", json={
            "model": "local-model",
            "messages": [
                {"role": "system", "content": "You are a helpful lung cancer diagnosis assistant."},
                {"role": "user", "content": prompt}
            ],
            "temperature": 0.7
        })
        return response.json()['choices'][0]['message']['content']
    except Exception as e:
        return f"Error contacting local LLM: {e}"

# -------- Log Date using sklearn (for demonstration, using pandas here) --------
def log_prediction(prediction):
    now = datetime.now()
    log_entry = pd.DataFrame([[now.strftime("%Y-%m-%d %H:%M:%S"), prediction]], columns=["timestamp", "prediction"])
    if os.path.exists(data_log_file):
        log_entry.to_csv(data_log_file, mode='a', header=False, index=False)
    else:
        log_entry.to_csv(data_log_file, index=False)

# -------- View Logs --------
def view_logs():
    if os.path.exists(data_log_file):
        st.subheader("🗂️ Prediction History")
        df = pd.read_csv(data_log_file)
        st.dataframe(df)
    else:
        st.info("No predictions logged yet.")

# -------- Streamlit App --------
st.set_page_config(page_title="Lung Cancer Assistant", layout="centered")
st.title("🫁 Lung Cancer Prediction + AI Assistant")

menu = st.sidebar.radio("Go to:", ["🏠 Image Prediction", "🤖 Ask AI (LLM)", "🗂️ View Logs", "📋 About"])

if menu == "🏠 Image Prediction":
    st.header("Upload a Chest CT Image")
    uploaded_file = st.file_uploader("Choose an image", type=["jpg", "jpeg", "png"])

    if uploaded_file is not None:
        image = Image.open(uploaded_file).convert("L")
        st.image(image, caption="Uploaded Image", use_column_width=True)
        if st.button("Predict"):
            result = predict_image(image)
            log_prediction(result)
            st.success(f"🧠 Prediction: **{result.upper()}**")

elif menu == "🤖 Ask AI (LLM)":
    st.header("AI Assistant for Lung Cancer Questions")
    question = st.text_area("Ask a question about lung cancer symptoms, risks, or treatment:")
    if st.button("Get Answer"):
        with st.spinner("Thinking..."):
            response = ask_local_llm(question)
        st.text_area("AI Response:", response, height=250)

elif menu == "🗂️ View Logs":
    view_logs()

elif menu == "📋 About":
    st.markdown("""
    ### About This App
    This app was built as a Personal Project by **Amogh Holla**. It consists of:

    - ✅ A **deep learning model** trained on CT scan images to detect lung cancer (benign / malignant / normal)
    - 💬 A **locally running LLM (LLaMA via LM Studio)** that provides health insights and guidance
    - 🛠️ Built with **PyTorch**, **Streamlit**, and **LM Studio**
    - 📅 Logs all predictions by date
    - 📊 Viewable history of predictions

    > Powered by the IQ-OTH/NCCD Lung Cancer Dataset
    """)
